In [1]:
import torch
import pytorch_lightning as L
from multimolecule import RnaTokenizer
import utrlm

torch.set_float32_matmul_precision('high')

/opt/miniconda3/envs/torch_cuda_128/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
utrlm_base_model = "multimolecule/utrlm-te_el"
utrlm_ckpt = "/Users/mkamruz/Public/SNL/Projects/5UTR/UTRLM_Ckpt/cao2021/epoch=05-val_loss=0.459-v1.ckpt"
test_data_csv = "/Users/mkamruz/Public/SNL/Projects/5UTR/Data/random_25mers_10M.csv"

utrlm_model = utrlm.UTRLMClassifier.load_from_checkpoint(utrlm_ckpt)
utrlm_model.eval()

utrlm_tokenizer = RnaTokenizer.from_pretrained(utrlm_base_model)

utrlm_dl = utrlm.DNADataModule(test_data=test_data_csv, 
                               batch_size=1024,
                               max_length=512, 
                               tokenizer=utrlm_tokenizer,
                               num_workers=10,
                               pin_memory=False
                               )
utrlm_dl.setup()
utrlm_dl = utrlm_dl.test_dataloader()

trainer = L.Trainer(
    accelerator="auto",
    devices=1,
    precision=32
)

Some weights of UtrLmForSequencePrediction were not initialized from the model checkpoint at multimolecule/utrlm-te_el and are newly initialized: ['sequence_head.decoder.bias', 'sequence_head.decoder.weight', 'utrlm.pooler.dense.bias', 'utrlm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [5]:
try:
    utrlm_preds = trainer.predict(utrlm_model, utrlm_dl, return_predictions=True, )
    res = []
    for pred in utrlm_preds:
        res.extend(pred.tolist())
    len(res)
    
except Exception as e:
    print(f"Error during prediction: {e}")
    utrlm_preds = None

/opt/miniconda3/envs/torch_cuda_128/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.


Predicting DataLoader 0:   0%|          | 0/9766 [00:00<?, ?it/s]Error during prediction: Invalid buffer size: 16.00 GB
